In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Step 1: Load data from Yahoo Finance
symbol = 'AAPL'
start_date = '2023-01-01'
end_date = '2023-03-31'

df = yf.download(symbol, start=start_date, end=end_date)

# Step 2: Feature engineering
df['RETURNS'] = df['Close'].pct_change()
df['PRX_MA_ND'] = df['Close'].rolling(window=5).mean()
df['VOLATILITY'] = df['Close'].rolling(window=5).std()
df['TP1_RETURNS'] = df['RETURNS'].shift(-1)
df['VOLUME'] = df['Volume']

# Step 3: Drop NaN values
df.dropna(inplace=True)

# Step 4: Split data into training and testing sets
df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
df_y = df['TP1_RETURNS']

if len(df_x) == 0 or len(df_y) == 0:
    raise ValueError("Empty dataset. Check if the data has any missing values.")

df_x_scaler = MinMaxScaler()
df_x_scaled = pd.DataFrame(df_x_scaler.fit_transform(df_x))

x_train, x_test, y_train, y_test = train_test_split(df_x_scaled,
                                                    df_y,
                                                    test_size=0.25,
                                                    random_state=42)

# Step 5: Reshape data for LSTM model
x_train = x_train.values.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.values.reshape((x_test.shape[0], 1, x_test.shape[1]))

# Step 6: Build LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(1))

model.compile(loss='mae', optimizer='adam')

# Step 7: Train LSTM model
history = model.fit(x_train, y_train, epochs=50, batch_size=72, validation_data=(x_test, y_test), verbose=1, shuffle=False)

# Step 8: Make predictions
y_pred = model.predict(x_test)

# Step 9: Inverse transform data to get actual prices
y_test_unscaled = df_x_scaler.inverse_transform(np.hstack((x_test.reshape(x_test.shape[0], x_test.shape[2]), y_test.reshape(y_test.shape[0], 1))))
y_pred_unscaled = df_x_scaler.inverse_transform(np.hstack((x_test.reshape(x_test.shape[0], x_test.shape[2]), y_pred)))

# Step 10: Plot results
plt.plot(y_test_unscaled[:, -1], label='Actual')
plt.plot(y_pred_unscaled[:, -1], label='Predicted')
plt.legend()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Anaconda\\lib\\site-packages\\scipy.libs\\.load-order-scipy-1.10.1'